# 6.1 Pig

## Что такое Pig

<img src="slides/le06/pig/pig-03.png" title="Что такое Pig" width="400" height="400"/>

## Для чего нужен Pig

<img src="slides/le06/pig/pig-04.png" title="Для чего нужен Pig" width="400" height="400"/>

## Основные возможности Pig

<img src="slides/le06/pig/pig-06.png" title="Основные возможности Pig" width="400" height="400"/>

## Основные возможности Pig

<img src="slides/le06/pig/pig-07.png" title="КОмпоненты Pig" width="400" height="400"/>

## Режим выполнения

<img src="slides/le06/pig/pig-08.png" title="Режим выполнения" width="400" height="400"/>

## Запуск Pig

<img src="slides/le06/pig/pig-09.png" title="Запуск Pig" width="400" height="400"/>

## Pig Latin

<img src="slides/le06/pig/pig-10.png" title="Pig Latin" width="400" height="400"/>

Pig - типизированный язык. 
- **field** - минимальный блок (число, строка и тд). 
- Объединение field - **tuple** (кортеж), заключенный в `(` и `)`. 
- **Bag** (мешок) - набор кортежей, заключенных в `{` и `}`. Кортежи могут отличаться по своему наполнению.

<img src="slides/le06/pig/pig-11.png" title="Pig Latin" width="400" height="400"/>

Схожесть с реляционной моделью:
- Bag vs Таблица БД
- Tuple vs Строка таблицы БД
    - каждый tuple Bag **может имеет различную** структуру
    - каждая строка БД **имеет одинаковую** структуру
- Field vs Поля строки

## Операции Dump и Store

<img src="slides/le06/pig/pig-13.png" title="Операции Dump и Store" width="400" height="400"/>

Работает режим отложенного запуска. Данные обрабатываются не в момент написания команд, а в момент необходимости вывода. Компилятор следит за синтаксисом.

## Большой объем данных

<img src="slides/le06/pig/pig-14.png" title="Большой объем данных" width="400" height="400"/>

## Операция Load

<img src="slides/le06/pig/pig-15.png" title="Операция Load" width="400" height="400"/>

По-умолчанию в using используется функция `PigStorage`, которая разделяет строки по табуляции. Можно изменить разделитель указав `PigStorage(';')`.

# 6.2 Основные операторы PigLatin

<img src="slides/le06/pig/pig-18.png" title="Пример" width="400" height="400"/>

- Сначала загружаем данные из `b.txt` файла в переменную `chars`. Файл предположительно состоит из одного поля `c`. 
- Вызываем `DESCRIBE` чтобы посмотреть описание переменной.
- Вызываем `DUMP`, чтобы посмотреть, что содержит переменная.
- Делаем группировку `GROUP`, указав данные какой переменной мы хотим сгруппировать и по какому полю.
- Вызываем `DESCRIBE` чтобы посмотреть описание переменной после группировки. Получаем некий bag, который содержит первым полем строку (значение ключа), второе поле тоже bag, который содержит все tuples из bag `chars`.

<img src="slides/le06/pig/pig-19.png" title="Пример" width="400" height="400"/>

`ILLUSTRATE`

## FOREACH

<img src="slides/le06/pig/pig-21.png" title="FOREACH" width="400" height="400"/>

`FOREACH <bag> GENERATE <data>` - перебирает все кортежи из некоторого bag и определяем, что делать с данными через `GENERATE`.

```
# Получаем все поля i из кортежа
counts = FOREACH records GENERATE i;
```

<img src="slides/le06/pig/pig-22.png" title="FOREACH с функцией" width="400" height="400"/>

Обычно `FOREACH` используют с функцией. В Pig стандартные функции:
- `COUNT` количество объектов
- `FLATTEN` 
- `CONCAT`.

Можно и пользовательские функции реализовывать.

<img src="slides/le06/pig/pig-23.png" title="Пример FOREACH с функцией" width="400" height="400"/>

Считаем количество объектов.

## TOKENIZE

<img src="slides/le06/pig/pig-24.png" title="TOKENIZE" width="400" height="400"/>

Функция разбивает строку на токены.

<img src="slides/le06/pig/pig-25.png" title="Пример TOKENIZE" width="400" height="400"/>

Зачитываем файл `c.txt` как одно большое поле `c`. Далее перебираем эти данные и применяем к каждой строке функцию `TOKENIZE` (аналогия со `split` строки). Было поле `(line)`, а мы его разбили на bag `{(token), (token), (token)}`.

**Дополнение**: запятые `TOKENIZE` отбрасывает. Он делит по пробелам, кавычкам, скобкам и запятым и звездочкам.

```
# Veni, vidi, vici
grunt> text =  LOAD 'latin.txt' AS (line:chararray);
grunt> tokens = FOREACH text GENERATE TOKENIZE(line);
grunt> DUMP tokens;

({(veni), (vidi), (vici)})
```

<img src="slides/le06/pig/pig-26.png" title="Пример TOKENIZE" width="400" height="400"/>

Чтобы обратно собрать все токены используют функцию `flatten`. Из `{(t), (t), (t)}` -> `(f) (f) (f)`. `flatten` принимает индекс строки. `$0` это индекс поля в кортеже.

## Пример решения WordCount на Pig

<img src="slides/le06/pig/pig-27.png" title="Пример WordCount" width="400" height="400"/>

## JOINS

<img src="slides/le06/pig/pig-28.png" title="JOINS" width="400" height="400"/>

<img src="slides/le06/pig/pig-29.png" title="INNER JOINS" width="400" height="400"/>

`JOIN posts BY user, likes BY user` мы смотрим пересечение таблицы `posts` по полю `user` с таблицей `likes` то же по полю `user`.

<img src="slides/le06/pig/pig-30.png" title="INNER JOINS" width="400" height="400"/>

Результат объединения таков, что сначала будут идти данные из 1 файла для **user1**, а потом из второго. **User5** есть только во 2 файле, поэтому информацию по нему в итоговое пересечение не попадет. **User3** есть только в 1 файле, поэтому данные тоже не попадут.

<img src="slides/le06/pig/pig-31.png" title="Структура данных INNER JOINS" width="400" height="400"/>

Так как мы имеем 2 поля **user**, то необходимо указывать префикс, чтобы обозначить к какому полю мы обращаемся.

<img src="slides/le06/pig/pig-32.png" title="OUTER JOINS" width="400" height="400"/>

- Поле из 1 источника остается в любом случае
- Поле из 2 источника остается в любом случае
- Поля из обеих полей остаются в любом случае

<img src="slides/le06/pig/pig-33.png" title="LEFT JOINS" width="400" height="400"/>

<img src="slides/le06/pig/pig-34.png" title="LEFT JOINS" width="400" height="400"/>

# 6.3 Hive

<img src="slides/le06/hive/hive-02.png" title="Hive" width="400" height="400"/>

Hive похож на Pig. Они используются для одних и тех же целей. Hive работает поверх Hadoop. Hive не БД.

Hive online:  
https://demo.gethue.com/hue/editor/?type=hive

<img src="slides/le06/hive/hive-03.png" title="Hive" width="400" height="400"/>

SQL-подобный язык HiveQL.

<img src="slides/le06/hive/hive-04.png" title="Hive" width="400" height="400"/>

Low latency запросы, которые предоставляются достаточно быстро. Хорошо масштабируемая система: хорошо работает как на малых объемах, так и на больших.

## Общая схема работы Hive

<img src="slides/le06/hive/hive-05.png" title="Общая схема работы Hive" width="400" height="400"/>

Пользователь пишет некоторый скрипт на HiveQL. Client Hive транслирует задачу в MapReduce задачи и выполняет на кластере Hadoop. По выполнению задачи результ отправляется обратно клиенту.

## Общая архитектура Hive

<img src="slides/le06/hive/hive-07.png" title="Общая архитектура Hive" width="400" height="400"/>

Центральное место занимает ядро **Hive**, которое состоит из **Query Parser** (раскладывает команды, которые клиент отрпавил для запуска). **Executor** запускает MapReduce задачи на кластере Hadoop.

В отличии от Pig, Hive использует фиксированную структуру данных и meta информацию он хранит в некотором хранилище **Metastore** (реляционная БД). 

Клиент может взаимодействовать через CLI, через Java API, web интерфейс. Обычно клиенты выполняют 2 типа действий:
- Определяют структуру и взаимодействуют с Metastore.
- Запуск происходит с помощью обращения к ядру, которое распарсит запрос и транслирует в MapReduce задачи.  

## Модель данных Hive

<img src="slides/le06/hive/hive-09.png" title="Модель данных Hive" width="400" height="400"/>

Модель данных очень похожа на обычную реляционную. В отличии от Pig строка имеет определенный набор полей. В начале пользователь должен описать структуру таблицы.

## Пример

<img src="slides/le06/hive/hive-10.png" title="Пример задачи Hive" width="400" height="400"/>

<img src="slides/le06/hive/hive-11.png" title="Пример задачи Hive" width="400" height="400"/>

Выполняем команду `hive`, чтобы войти в оболочку. В Hive мы можем выполнять стандартные Linux команды, только мы должны указать `!cat`.

<img src="slides/le06/hive/hive-12.png" title="Пример задачи Hive" width="400" height="400"/>

Далее описываем структуру таблицы. 
- `CREATE TABLE post ...` - объявление таблицы
- `ROW FORMAT DELIMITED`
    - `FIELDS TERMINATED BY ','` разделитель
    - `STORED AS TEXTFILE` как сохранять данные

```sql
CREATE TABLE posts 
(
    user STRING,
    post STRING,
    time BIGINT
)
ROW FORMAT DELIMITED
FIELDS TERMITED BY ','
STORED AS TEXTFILE;
```

```bash
# Какие таблицы есть
show tables

# Описание таблицы
describe posts
```

<img src="slides/le06/hive/hive-14.png" title="Пример задачи Hive" width="400" height="400"/>

`select count(1) from posts` - считает количество записей в таблице posts. HiveQL преобразуется в 1 MapReduce задачу.


<img src="slides/le06/hive/hive-15.png" title="Пример задачи Hive" width="400" height="400"/>

Мы можем выбрать все записи пользователя:  
`select * from posts where user="user2"`

## Hive: Joins

<img src="slides/le06/hive/hive-19.png" title="Hive Joins" width="400" height="400"/>

По-умолчанию Inner Join. В отличие от Pig Hive умеет объединять несколько таблиц.

<img src="slides/le06/hive/hive-20.png" title="Пример Hive Join" width="400" height="400"/>

<img src="slides/le06/hive/hive-21.png" title="Пример Hive Join" width="400" height="400"/>

Заполняем таблицу **posts_likes** пересечением двух таблиц:
```SQL
INSERT OVERWRITE TABLE posts_likes
SELECT p.user, p.post, l.count
FROM posts p 
JOIN likes l
ON (p.user = l.user)
```

## Пример WordCounts

<img src="slides/le06/hive/hive-23.png" title="Пример Hive WordCounts" width="400" height="400"/>

   

Для подсчета слов используем временную таблицу
```sql
/* Сначала объявляем таблицу, в которой будем хранить весь текст в одной колонке типа STRING. */
CREATE TABLE docs (line STRING);

/*Загружаем данные из файла docks в таблицу docks путем перезаписи.*/
LOAD DATA INPATH 'docs' OVERWRITE INTO TABLE docs;

/*
Создаем временную таблицу word_counts (аналогия WITH).
Выбираем из нее слово и считаем количество
В подзапросе разбиваем строку на слова
explode - каждое слово, как отдельная строка
w - название подзапроса
*/
CREATE TABLE word_counts AS
SELECT word, 
       count(1) AS count 
FROM
    (SELECT explode(split(line, '\s')) AS word 
       FROM docs) w)
GROUP BY word
ORDER BY word;
```

<img src="slides/le06/hive/hive-24.png" title="Пример Hive WordCounts" width="400" height="400"/>

Hive для любителей SQL.